In [0]:
%sh

rm -rf /Volumes/edsm/silver/utility/systemsPopulated/_checkpoints/

In [0]:
import os
import sys

sys.path.append(f"{os.getcwd()}/..")

In [0]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType
from pyspark.sql.functions import concat, regexp_extract, date_format, current_timestamp
from pyspark.sql.functions import when, col, to_timestamp, to_date, regexp_replace
from pyspark.sql.functions import sha2, concat_ws, coalesce, lit, trim, struct
from pyspark.sql.functions import to_json, expr, to_utc_timestamp
from pyspark.sql.types import StructType, ArrayType
from pyspark.sql.functions import transform
import re
from functions import create_table_if_not_exists, get_function, silver_scd2_transform
from functions import stream_read_table, stream_write_scd2_table
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

settings = {
    "read_function": "functions.stream_read_table",
    "transform_function": "functions.silver_scd2_transform",
    "write_function": "functions.stream_write_scd2_table",
    "upsert_function": "functions.scd2_upsert",
    "src_table_name": "edsm.bronze.systemsPopulated",
    "dst_table_name": "edsm.silver.systemsPopulated",
    "build_history": "true",
    "business_key": [
        "id"
    ],
    "surrogate_key": [
        "allegiance",
        "controllingFaction",
        "economy",
        "factions",
        "government",
        "population",
        "security",
        "state",
        "stations"
    ],
    "use_row_hash": True,
    "row_hash_col": "row_hash",
    "data_type_map": {
        "date": "timestamp"
    },
    "readStreamOptions": {},
    "writeStreamOptions": {
        "mergeSchema": "false",
        "checkpointLocation": "/Volumes/edsm/silver/utility/systemsPopulated/_checkpoints/",
        "delta.columnMapping.mode": "name"
    },
    "ingest_time_column": "derived_ingest_time"
}

df = stream_read_table(spark, settings)
df = silver_scd2_transform(df, settings, spark)

stream_write_scd2_table(df, settings, spark)